# labelmeのアノテーションデータがない画像データ用

# Library

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd

from PIL import Image
import matplotlib.pyplot as plt
import cv2

import tqdm


# define ImageDevider class 

In [5]:
# 推論用、画像分類用（アノテーションデータなし）
class ImageDevider():
    """
    画像分割プログラム
    アノテーションファイルが無くても動きます。
    分割後のファイルには「元のファイル名」+「二桁の連番」+「拡張子」となります。<-推論後に活用ください
    initializer args
    out_y: 出力のy軸ピクセル数
    out_x: 出力のx軸ピクセル数
    overlap: 切り取る時の隣とオーバーラップさせるピクセル数
    output_img_folder: 切り取った画像を保存するファイルパス
    """
    def __init__(self, out_y: int, out_x: int, overlap: int, output_img_folder: str) -> None:
        self.out_y = out_y
        self.out_x = out_x
        self.overlap = overlap
        self.output_img_folder = output_img_folder

    def devide_image(self, image_file: str) -> None:
        """
        arg1: image_file 画像ファイルへのパス
        """
        img = Image.open(image_file)
        # open images, get height & width
        img_array = np.array(img)
        in_y, in_x, _ = img_array.shape
        
        num_images_y = in_y // self.out_y
        num_images_x = in_x // self.out_x

        # if crop images are lack at righet & bottom, num of image +1
        if in_y - num_images_y * (self.out_y - self.overlap) > self.overlap:
            num_images_y += 1
        if in_x- num_images_x * (self.out_x - self.overlap) > self.overlap:
            num_images_x += 1

        basename = os.path.basename(image_file).split('.')[0]

        # set index to create save-file name
        index = 0

        # crop file to small size
        for i in range(num_images_y):
            for j in range(num_images_x):
                start_y = np.maximum(0, i * (self.out_y - self.overlap))
                start_x = np.maximum(0, j * (self.out_x - self.overlap))
                end_y = i * (self.out_y - self.overlap) + self.out_y
                end_x = j * (self.out_x - self.overlap) + self.out_x
                output_array = img_array[start_y: end_y, start_x: end_x]
                
                img = cv2.copyMakeBorder(output_array, 
                                   0, self.out_y - output_array.shape[0], 
                                   0, self.out_x - output_array.shape[1], 
                                   cv2.BORDER_CONSTANT, 
                                   0)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (640, 640))
                cv2.imwrite(self.output_img_folder + f'{basename}_{index:02}.png', img)
                index += 1

# 使い方

In [6]:
crop_man_train = ImageDevider(1280, 1280, 256, '/output_folder/')

In [8]:
crop_man_train.devide_image('./images/image001.jpeg')

100%|██████████| 193/193 [00:15<00:00, 12.20it/s]
